In [1]:
import pandas as pd
import helpers
import json
import keras
from keras.layers import Dense
import tensorflow as tf

In [2]:
train_df = pd.read_json('./data/train_dataset.jl', lines=True, chunksize=500)
test_df = pd.read_json('./data/test_dataset.jl', lines=True, chunksize=500)

In [3]:
#embeddings = helpers.load_embeddings()

In [4]:
items = helpers.load_items()
items_ids = [key for key in items]
categorical_items = [i for i in range(len(items_ids))]
item_categorical_map = {}
for i in range(len(items_ids)):
    item_categorical_map[items_ids[i]] = categorical_items[i]

In [5]:
def process(raw_data):
    new_df = pd.DataFrame()
    new_df['events'] = [x for x in raw_data['user_history']]
    #new['avg_time_between_visits'] =
    new_df['view_count'] = new_df['events'].apply(lambda x: sum(1 for y in x if y['event_type'] == 'view'))
    new_df['search_count'] = new_df['events'].apply(lambda x: sum(1 for y in x if y['event_type'] == 'search'))
    new_df['purchase_count'] = new_df['events'].apply(lambda x: sum(1 for y in x if y['event_type'] == 'purchase'))
    #new_df['main_category'] = 
    #print(raw_data)
    #new_df['avg_title']
    #new['view_count']
    #events_feartures = []
    target =  tf.constant([item_categorical_map[x] for x in raw_data['item_bought']], tf.int32) if 'item_bought' in raw_data.columns else None
    return new_df.drop(columns=['events']), target

In [6]:
next(train_df)

,user_history,item_bought
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207
...,...,...
495,"[{'event_info': 248216, 'event_timestamp': '20...",248216
496,"[{'event_info': 'IPHOBE7', 'event_timestamp': ...",746346
497,"[{'event_info': 'ESFOLIADOR', 'event_timestamp...",1886047
498,"[{'event_info': 'MAMELUCOS ADULTOS', 'event_ti...",341259


In [7]:
process(next(train_df))

(     view_count  search_count  purchase_count
 0             2             5               0
 1             7             1               0
 2             9            27               0
 3            17            25               0
 4             1             5               0
 ..          ...           ...             ...
 495          80             0               0
 496          10             4               0
 497          22            25               0
 498          27            46               0
 499          63            52               0
 
 [500 rows x 3 columns],
 <tf.Tensor: shape=(500,), dtype=int32, numpy=
 array([1964592,  461968,  357201,  584078, 2090537, 1431787,  212890,
        1180357, 1211029,  363635, 2036590, 1849816,  959616, 1875637,
        1218335, 1329125, 1609529,  771382, 1641360,  121521,  219130,
        1695316, 1385566,  741498,  224150, 1396445, 1745975,  726505,
         859618, 1679868,  588587, 1325331, 1348001, 1917462, 1293211,
       

In [7]:
def build_model():
    model = keras.models.Sequential()
    model.add(Dense(2, activation='relu'))
    model.add(Dense(len(items), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
%%time
model = build_model()
for df in train_df:
    features, target = process(df)
    model.fit(features, target, epochs=10)
    scores = model.evaluate(features, target)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    

Epoch 1/10
16/16 [==============================] - 12s 740ms/step - loss: 14.5663 - accuracy: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 12s 755ms/step - loss: 14.4936 - accuracy: 0.0020
Epoch 3/10
16/16 [==============================] - 11s 672ms/step - loss: 14.3798 - accuracy: 0.0040
Epoch 4/10
16/16 [==============================] - 11s 662ms/step - loss: 14.2124 - accuracy: 0.0020
Epoch 5/10
16/16 [==============================] - 11s 669ms/step - loss: 13.9779 - accuracy: 0.0040
Epoch 6/10
16/16 [==============================] - 11s 706ms/step - loss: 13.6635 - accuracy: 0.0040
Epoch 7/10
16/16 [==============================] - 11s 686ms/step - loss: 13.2832 - accuracy: 0.0040
Epoch 8/10
16/16 [==============================] - 13s 806ms/step - loss: 12.8571 - accuracy: 0.0040
Epoch 9/10
16/16 [==============================] - 12s 749ms/step - loss: 12.4264 - accuracy: 0.0060
Epoch 10/10
16/16 [==============================] - 11s 666ms/step - loss: 11

KeyboardInterrupt: 

In [9]:
model.test

AttributeError: 'Sequential' object has no attribute 'test'

In [43]:
df = next(train_df)
df

,user_history,item_bought
2000,[{'event_info': 'CABECOTE CUMMINS SERIE C 220C...,1553366
2001,"[{'event_info': 'PAINEL MAQUINA BRASTEMP 8', '...",1756706
2002,"[{'event_info': 439642, 'event_timestamp': '20...",26285
2003,"[{'event_info': 1831446, 'event_timestamp': '2...",175411
2004,"[{'event_info': 'CAPOTA MARITIMA', 'event_time...",1914101
...,...,...
2495,"[{'event_info': 366604, 'event_timestamp': '20...",1371302
2496,"[{'event_info': 'TENIS COCA COLA', 'event_time...",947444
2497,[{'event_info': 'PUERTA CRISTAL TEMPLADO DOBLE...,978261
2498,"[{'event_info': 1105872, 'event_timestamp': '2...",1105872


In [9]:
features = process(next(train_df))

In [40]:
item_dict

{371737: {'title': 'Guantes De Box  Everlast 8 12 14 16 Onzas Negro  Rojo  Rosas',
  'domain_id': 'MLM-MARTIAL_ARTS_AND_BOXING_EQUIPMENT',
  'product_id': nan,
  'price': 779.0,
  'category_id': 'MLM2483',
  'condition': 'new'},
 372805: {'title': 'Grupo Shimano/sunrace Para Bicicleta Mtb Completo ',
  'domain_id': 'MLM-BICYCLE_GROUPSETS',
  'product_id': nan,
  'price': 2000.0,
  'category_id': 'MLM429290',
  'condition': 'new'},
 1275420: {'title': 'Torre Paletera Mdf Mesa Dulces Candy Bar Candybar Paletero',
  'domain_id': 'MLM-PARTY_SUPPLIES',
  'product_id': nan,
  'price': 190.0,
  'category_id': 'MLM179575',
  'condition': 'new'},
 2077354: {'title': 'Vinagre Balsámico Pianello Italiano De 1 Litro',
  'domain_id': 'MLM-BALSAMIC_VINEGARS',
  'product_id': nan,
  'price': 97.85,
  'category_id': 'MLM191658',
  'condition': 'new'},
 1803528: {'title': 'Camisa Vintage Hombre Casual Manga Larga Paisley',
  'domain_id': 'MLM-SHIRTS',
  'product_id': nan,
  'price': 552.39,
  'category